**Agent 1 (Razumevanje korisničkog pitanja)**

In [1]:
import spacy
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import json

nlp_spacy = spacy.load("en_core_web_sm")

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model1 = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
ner_pipeline = pipeline("ner", model=model1, tokenizer=tokenizer, aggregation_strategy="simple")

def analyze_user_query_advanced(query):
    doc = nlp_spacy(query)
    keywords = [chunk.text for chunk in doc.noun_chunks]

    ner_results = ner_pipeline(query)

    combined_keywords = list(set(keywords))

    if "what" in query.lower():
        question_type = "definition"
    elif "how" in query.lower():
        question_type = "explanation"
    elif "why" in query.lower():
        question_type = "analysis"
    elif "when" in query.lower():
        question_type = "time"
    elif "compare" in query.lower() or "difference" in query.lower():
        question_type = "comparison"
    elif "list" in query.lower() or "examples" in query.lower():
        question_type = "list"
    else:
        question_type = "general"

    result = {
        "original_query": query,
        "keywords": combined_keywords,
        "question_type": question_type
    }
    return result

query = "Tell me about architecture of fully connected neural networks"
response = analyze_user_query_advanced(query)
print(json.dumps(response, indent=2))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{
  "original_query": "Tell me about architecture of fully connected neural networks",
  "keywords": [
    "fully connected neural networks",
    "architecture",
    "me"
  ],
  "question_type": "general"
}


**Agent 2 (Pretraga informacija)**

In [2]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.3 MB/s eta 0:00:00


In [3]:
import pdfplumber
from sentence_transformers import SentenceTransformer, util
import json

model2 = SentenceTransformer('all-MiniLM-L6-v2')

def search_pdf_with_context(file_path, keywords, query):
    results = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            sentences = text.split('.')

            for sentence in sentences:
                if any(keyword.lower() in sentence.lower() for keyword in keywords):
                    results.append({
                        "page_number": page.page_number,
                        "sentence": sentence.strip()
                    })

    if query and results:
        sentences = [result["sentence"] for result in results]
        query_embedding = model2.encode(query, convert_to_tensor=True)
        sentence_embeddings = model2.encode(sentences, convert_to_tensor=True)

        similarities = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]
        ranked_results = sorted(
            zip(similarities, results), key=lambda x: x[0], reverse=True
        )
        results = [{"score": float(sim.item()), **res} for sim, res in ranked_results]

    return results

def second_agent(input_data):
    keywords = input_data.get("keywords", [])
    pdf_path = input_data.get("pdf_path", "example.pdf")
    query = input_data.get("original_query", "")
    question_type = input_data.get("question_type", "general")

    search_results = search_pdf_with_context(pdf_path, keywords, query)

    result = {
        "original_query": query,
        "question_type": question_type,
        "search_results": search_results
    }
    return result

first_agent_output = {
    "original_query": "What are the applications of CNNs?",
    "keywords": ["CNNs", "applications"],
    "question_type": "explanation",
    "pdf_path": "projekat2.pdf"
}
response = second_agent(first_agent_output)
print(json.dumps(response, indent=2))




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{
  "original_query": "What are the applications of CNNs?",
  "question_type": "explanation",
  "search_results": [
    {
      "score": 0.6647508144378662,
      "page_number": 4,
      "sentence": "Key components of CNNs include:\n\uf0b7 Convolutional Layers: Use filters (kernels) to scan the input, extracting spatial\nfeatures like edges and textures"
    },
    {
      "score": 0.6171005964279175,
      "page_number": 4,
      "sentence": "Convolutional Neural Networks (CNNs)\nConvolutional Neural Networks (CNNs) are specialized neural networks designed for\nprocessing data with a grid-like topology, such as images"
    },
    {
      "score": 0.6143110990524292,
      "page_number": 3,
      "sentence": "Some prominent applications include:\n\uf0b7 Computer Vision: Object detection, facial recognition, medical imaging, and\nautonomous vehicles"
    },
    {
      "score": 0.5894182920455933,
      "page_number": 4,
      "sentence": "Extensions\nof CNNs, such as ResNet, Inception,

**Agent 3 (Generisanje odgovora):**

In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model3 = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

def clean_text(text):
    import re
    text = text.replace("\n", " ").replace("\uf0b7", "")
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return text.strip()

def generate_answer_with_bart(input_data):
    search_results = input_data.get("search_results", [])
    original_query = input_data.get("original_query", "")

    score_threshold = 0.5
    relevant_passages = [
        clean_text(result["sentence"]) for result in search_results if result["score"] >= score_threshold
    ]

    if not relevant_passages:
        return {
            "original_query": original_query,
            "generated_answer": "Sorry, no relevant information was found to generate an answer.",
            "used_passages": []
        }

    combined_text = " ".join(relevant_passages)
    prompt = f"""
    User question: "{original_query}"

    Relevant information: {combined_text}

    Answer concisely and accurately based on the provided information.
    """

    inputs = tokenizer(
        "summarize: " + prompt,
        return_tensors="pt",
        max_length=1024,
        padding="max_length",
        truncation=True
    )

    summary_ids = model3.generate(
        inputs["input_ids"],
        max_length=400,
        min_length=80,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    generated_answer = tokenizer.decode(summary_ids[0], skip_special_tokens=True).strip()

    removal_phrases = [
        "Answer the user's question concisely and accurately based on the provided information.",
        "summarize:"
    ]
    for phrase in removal_phrases:
        generated_answer = generated_answer.replace(phrase, "").strip()

    return {
        "original_query": original_query,
        "generated_answer": generated_answer,
        "used_passages": relevant_passages
    }

second_agent_output = {
  "original_query": "What are the applications of CNNs?",
  "question_type": "explanation",
  "search_results": [
    {
      "score": 0.6647508144378662,
      "page_number": 4,
      "sentence": "Key components of CNNs include:\n\uf0b7 Convolutional Layers: Use filters (kernels) to scan the input, extracting spatial\nfeatures like edges and textures"
    },
    {
      "score": 0.6171005964279175,
      "page_number": 4,
      "sentence": "Convolutional Neural Networks (CNNs)\nConvolutional Neural Networks (CNNs) are specialized neural networks designed for\nprocessing data with a grid-like topology, such as images"
    },
    {
      "score": 0.6143110990524292,
      "page_number": 3,
      "sentence": "Some prominent applications include:\n\uf0b7 Computer Vision: Object detection, facial recognition, medical imaging, and\nautonomous vehicles"
    },
    {
      "score": 0.5894182920455933,
      "page_number": 4,
      "sentence": "Extensions\nof CNNs, such as ResNet, Inception, and EfficientNet, have further improved\nperformance and efficiency in computer vision tasks"
    },
    {
      "score": 0.5653724670410156,
      "page_number": 4,
      "sentence": "CNNs excel due to their ability to capture hierarchical patterns in images, starting from\nlow-level features (edges) to high-level representations (shapes or objects)"
    },
    {
      "score": 0.5324292182922363,
      "page_number": 3,
      "sentence": "Advanced Architectures in Deep Learning\nDeep learning has given rise to several advanced neural network architectures tailored\nfor specific problems:\n\uf0b7 Convolutional Neural Networks (CNNs): Used for image processing tasks"
    }
  ]
}
response = generate_answer_with_bart(second_agent_output)
print(response)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

{'original_query': 'What are the applications of CNNs?', 'generated_answer': 'Convolutional Neural Networks (CNNs) are specialized neural networks designed for processing data with a grid-like topology, such as images. CNNs excel due to their ability to capture hierarchical patterns in images, starting from low-level features (edges) to high-level representations (shapes or objects) Some prominent applications include: Computer Vision, facial recognition, medical imaging, and autonomous vehicles.', 'used_passages': ['Key components of CNNs include: Convolutional Layers: Use filters (kernels) to scan the input, extracting spatial features like edges and textures', 'Convolutional Neural Networks (CNNs) Convolutional Neural Networks (CNNs) are specialized neural networks designed for processing data with a grid-like topology, such as images', 'Some prominent applications include: Computer Vision: Object detection, facial recognition, medical imaging, and autonomous vehicles', 'Extensions 

**Agent 4 (Korisnička konverzacija)**

In [5]:
from sentence_transformers import SentenceTransformer, util
import json

similarity_model = SentenceTransformer('all-MiniLM-L6-v2')

interaction_history = []

def compute_similarity(query_embedding, previous_responses):
    if not previous_responses:
        return 0.0, None

    response_embeddings = similarity_model.encode(previous_responses, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, response_embeddings)[0]
    max_similarity = float(similarities.max())
    most_similar_idx = int(similarities.argmax())

    return max_similarity, most_similar_idx

def fourth_agent(input_query):
    global interaction_history

    query_embedding = similarity_model.encode(input_query, convert_to_tensor=True)

    previous_responses = [entry['response'] for entry in interaction_history]

    similarity, idx = compute_similarity(query_embedding, previous_responses)

    similarity_threshold = 0.7

    if similarity >= similarity_threshold:
        context = interaction_history[idx]
        combined_prompt = f"User asked: {context['query']} Follow-up: {input_query}"
        print(f"[INFO]: Pitanje je povezano sa prethodnim. Proširujem kontekst:\n{combined_prompt}")
        response = generate_answer_with_bart({
            "original_query": combined_prompt,
            "search_results": context['search_results']
        })['generated_answer']
    else:
        print("[INFO]: Pitanje nije povezano sa istorijom. Pokrećem novu analizu...")
        first_agent_output = analyze_user_query_advanced(input_query)
        first_agent_output["pdf_path"] = "projekat2.pdf"
        second_agent_output = second_agent(first_agent_output)

        if not second_agent_output.get("search_results"):
            return "Sorry, no relevant information was found."

        response = generate_answer_with_bart(second_agent_output)['generated_answer']

    interaction_history.append({
        "query": input_query,
        "response": response,
        "search_results": second_agent_output.get("search_results", [])
    })

    return response


**PRIMOPREDAJA**

In [6]:
def interactive_system():
    print("Dobrodošli u sistem za istraživanje! 😊")
    print("Unesite vaše pitanje ili 'exit' za izlaz.")

    while True:
        user_query = input("\nUnesite vaše pitanje: ").strip()
        if user_query.lower() == "exit":
            print("Hvala na korišćenju sistema! Do viđenja! 👋")
            break

        print("\n[Prvi agent]: Analiziram pitanje...")
        try:
            first_agent_output = analyze_user_query_advanced(user_query)
        except Exception as e:
            print(f"[ERROR]: Došlo je do greške u Prvom agentu: {str(e)}")
            continue

        first_agent_output["pdf_path"] = "projekat2.pdf"

        print("\n[Drugi agent]: Pretražujem relevantne informacije u PDF-u...")
        try:
            second_agent_output = second_agent(first_agent_output)
            if not second_agent_output.get("search_results"):
                print("[INFO]: Nisu pronađene relevantne informacije.")
                continue
        except Exception as e:
            print(f"[ERROR]: Došlo je do greške u Drugom agentu: {str(e)}")
            continue

        print("\n[Treći agent]: Generišem smisleni odgovor...")
        try:
            final_response = generate_answer_with_bart(second_agent_output)
            print("\nKonačan odgovor:", final_response["generated_answer"])
        except Exception as e:
            print(f"[ERROR]: Došlo je do greške u Trećem agentu: {str(e)}")

        print("\nAko imate novo pitanje, unesite ga ili unesite 'exit' za izlaz.")


In [8]:
interactive_system()

Dobrodošli u sistem za istraživanje! 😊
Unesite vaše pitanje ili 'exit' za izlaz.

Unesite vaše pitanje: Tell me about machine learning?

[Prvi agent]: Analiziram pitanje...

[Drugi agent]: Pretražujem relevantne informacije u PDF-u...

[Treći agent]: Generišem smisleni odgovor...

Konačan odgovor: Machine Learning (ML) is a branch of artificial intelligence (AI) that focuses on developing algorithms and statistical models. ML has numerous applications, such as natural language processing (NLP), computer vision, recommendation systems, fraud detection, and medical diagnosis. Machine learning is broadly categorized into three main types based on how models learn from data: 1 Deep Learning: Algorithms inspired by the structure and function of the human brain, known as artificial neural networks. 2 Feature Learning: Unlike traditional machine learning, DL models learn features directly from raw data.

Ako imate novo pitanje, unesite ga ili unesite 'exit' za izlaz.

Unesite vaše pitanje: An